In [1]:
import findspark

In [2]:
# Spark-Kafka Integration is supported in version 2.7 and above. so make sure you are 
# using higher spark version. 
# In case you are using earlier version and for this streaming purpose, 
# you want to point to hadoop
# can be done by passing parameter to .init() method

findspark.init("d:\\spark3")

In [3]:
from pyspark.sql import SparkSession

In [4]:
# Make sure right jar is included as for spark-sql-kafka integration

spark = SparkSession \
    .builder \
    .appName("kAFKA Streaming Demo") \
    .master("local[3]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .getOrCreate()

In [5]:
spark

In [6]:
from pyspark.sql.functions import from_json, expr
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType
from pyspark.sql import functions as F


In [7]:
schema = StructType([
        StructField("InvoiceNumber", StringType()),
        StructField("CreatedTime", LongType()),
        StructField("StoreID", StringType()),
        StructField("PosID", StringType()),
        StructField("CashierID", StringType()),
        StructField("CustomerType", StringType()),
        StructField("CustomerCardNo", StringType()),
        StructField("TotalAmount", DoubleType()),
        StructField("NumberOfItems", IntegerType()),
        StructField("PaymentMethod", StringType()),
        StructField("CGST", DoubleType()),
        StructField("SGST", DoubleType()),
        StructField("CESS", DoubleType()),
        StructField("DeliveryType", StringType()),
        StructField("DeliveryAddress", StructType([
            StructField("AddressLine", StringType()),
            StructField("City", StringType()),
            StructField("State", StringType()),
            StructField("PinCode", StringType()),
            StructField("ContactNumber", StringType())
        ])),
        StructField("InvoiceLineItems", ArrayType(StructType([
            StructField("ItemCode", StringType()),
            StructField("ItemDescription", StringType()),
            StructField("ItemPrice", DoubleType()),
            StructField("ItemQty", IntegerType()),
            StructField("TotalValue", DoubleType())
        ]))),
    ])

In [8]:
kafka_df = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "invoices")\
        .option("startingOffsets", "earliest")\
        .load()

In [9]:
value_df = kafka_df.select(from_json(F.col("value").cast("string"), schema).alias("value"))

In [10]:
explode_df = value_df.selectExpr("value.InvoiceNumber", "value.CreatedTime", "value.StoreID",
                                     "value.PosID", "value.CustomerType", "value.PaymentMethod", "value.DeliveryType",
                                     "value.DeliveryAddress.City",
                                     "value.DeliveryAddress.State", "value.DeliveryAddress.PinCode",
                                     "explode(value.InvoiceLineItems) as LineItem")

In [11]:
 flattened_df = explode_df \
        .withColumn("ItemCode", expr("LineItem.ItemCode")) \
        .withColumn("ItemDescription", expr("LineItem.ItemDescription")) \
        .withColumn("ItemPrice", expr("LineItem.ItemPrice")) \
        .withColumn("ItemQty", expr("LineItem.ItemQty")) \
        .withColumn("TotalValue", expr("LineItem.TotalValue")) \
        .drop("LineItem")

In [12]:
invoice_writer_query = flattened_df.writeStream \
        .format("json") \
        .queryName("Flattened Invoice Writer") \
        .outputMode("append") \
        .option("path", "d:\\demo\\source-kafka\\output") \
        .option("checkpointLocation", "d:\\demo\\source-kafka\\chk-point-dir") \
        .trigger(processingTime="1 minute") \
        .start()

In [ ]:
invoice_writer_query.awaitTermination()